### Code of paper

In [ ]:
import pandas as pd 
import numpy as np
import os
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import spotpy
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from matplotlib import rcParams
from matplotlib import rc
font = {'family' : 'serif'}
rc('mathtext', default='regular')
rc('font', **font)
rc('xtick', labelsize=12)
rc('ytick', labelsize=12)
rc('axes',labelsize = 12)

In [ ]:
# 删除异常值，并使用 np.nan 填充
def outlier_rev(data,repl = True,upper = 0.9987,lower = 0.0013):
    df = data.copy()
    min = df.quantile(lower)
    max = df.quantile(upper)
    if(repl):
        df[df > max] = np.nan
        df[df < min] = np.nan
        df1 = df
    else:
        df1 = df[df < max]
        df1 = df1[df1>min]    
    return df1

In [ ]:
def r2(x, y,fit_intercept=True):
    nas = np.logical_or(outlier_rev(x).isnull(), outlier_rev(y).isnull())  
    r = stats.pearsonr(x[~nas], y[~nas])[0]**2
    p  = stats.pearsonr(x[~nas], y[~nas])[1]
    x = x[~nas].values.reshape(-1,1)
    y = y[~nas].values.reshape(-1,1)
    regr = linear_model.LinearRegression(fit_intercept = fit_intercept)
    regr.fit(x,y)
    y_pred = regr.predict(x)
    
    rms = np.sqrt(mean_squared_error(y,y_pred))
    rmse = np.sqrt(mean_squared_error(x,y))
    return r,p,regr.coef_[0],regr.intercept_

def nonlin(x,a,b,c):
    return (a * x + b)/(x + c)

def damm(x,y):
    nas = np.logical_or(outlier_rev(x).isnull(), outlier_rev(y).isnull())
    x_ = x[~nas].values
    y_ = y[~nas].values
    popt, pcov = curve_fit(nonlin, x_, y_)
#     print(popt)
    y = nonlin(x, *popt)
    return y

In [ ]:
def calc_theta_s(xlat, xlong, doy, year, ftime):
    pid180 = np.pi / 180
    pid2 = np.pi / 2.0

    # Latitude computations
    xlat = np.radians(xlat)
    sinlat = np.sin(xlat)
    coslat = np.cos(xlat)

    # Declination computations
    kday = (year - 1977.0) * 365.0 + doy + 28123.0
    xm = np.radians(-1.0 + 0.9856 * kday)
    delnu = (2.0 * 0.01674 * np.sin(xm)
             + 1.25 * 0.01674 * 0.01674 * np.sin(2.0 * xm))
    slong = np.radians((-79.8280 + 0.9856479 * kday)) + delnu
    decmax = np.sin(np.radians(23.44))
    decl = np.arcsin(decmax * np.sin(slong))
    sindec = np.sin(decl)
    cosdec = np.cos(decl)
    eqtm = 9.4564 * np.sin(2.0 * slong) / cosdec - 4.0 * delnu / pid180
    eqtm = eqtm / 60.0

    # Get sun zenith angle
    timsun = ftime  # MODIS time is already solar time
    hrang = (timsun - 12.0) * pid2 / 6.0
    theta_s = np.arccos(sinlat * sindec + coslat * cosdec * np.cos(hrang))

    # if the sun is below the horizon just set it slightly above horizon
    theta_s = np.minimum(theta_s, pid2 - 0.0000001)
    theta_s = np.degrees(theta_s)

    return np.asarray(theta_s)

In [ ]:
# preprocessing

class predata(object):    
    def __init__(self,data,lat,lon):
        self.data = outlier_rev(data)
        self.lat = lat
        self.lon = lon
        
    def VPD(self):
        data = self.data
        data['VPD'] = (100 - data['Rh'])/100 * .6108 * np.exp(17.27*data['Ta']/(data['Ta']+237.3))
        return data
    
    def SZA(self):
        data = self.VPD()
        D_hour = data.index.hour + data.index.minute/60
        doy = data.index.dayofyear
        year = data.index.year
        data['SZA'] = np.deg2rad(calc_theta_s(self.lat,self.lon,doy,year,D_hour))
        return data
    
    def Ga(self):
        data = self.SZA()
        data['Ga'] = data['Ga']
        return data
    
    def Ac(self):
        data = self.Ga()
        # 冠层可用能量 （W/m^2）,Beer's law
        data['Ac'] = (1-np.exp(-0.5*data['LAI']/np.cos(data['SZA']))) * data['Rn']
#         data['Ac'] = (1-np.exp(-0.7*data['LAI'])) * data['Rn'] 
        return data

    def rhoa(self):
        data = self.Ac()
        # air mass density (kg/m3)
        data['rho_a'] =  3.486 *data['P']/(data['Ta']+273.15)/1.01
        return data

In [ ]:
os.chdir('D:\SIF\siteinput')

In [ ]:
Cp = 1013

In [ ]:
daman = pd.read_csv('dm_input.csv',index_col=[0],parse_dates=True)
huailai = pd.read_csv('hl_input.csv',index_col=[0],parse_dates=True)
niwot = pd.read_csv('nw_input.csv',index_col=[0],parse_dates=True)
harvard = pd.read_csv('hf_input.csv',index_col=[0],parse_dates=True)

In [ ]:
daman = predata(daman,38.8555,100.3722).rhoa().dropna()
huailai = predata(huailai, 40.35,115.79).rhoa().dropna()
niwot = predata(niwot,40.03,-105.55).rhoa().dropna()
harvard = predata(harvard,40.06,-88.2).rhoa().dropna()

In [ ]:
daman['Gamma'] = 0
huailai['Gamma'] = 0 
niwot['Gamma'] = 36.9 + 1.18*(niwot['Ta']-25) + 0.036*(niwot['Ta']-25)**2
harvard['Gamma'] = 36.9 + 1.18*(harvard['Ta']-25) + 0.036*(harvard['Ta']-25)**2

In [ ]:
# niwot = niwot[niwot['Ta'] > 0]
niwot['CO2'] = 380
niwot['GPP'] = niwot['GPP']/(0.0216*48)

In [ ]:
daman['site'] = 0
huailai['site'] = 1
niwot['site'] = 2
harvard['site'] = 3

In [ ]:
data = pd.concat([daman,huailai,niwot,harvard])

In [ ]:
data['h2o'] = data['LE']/44.100

# Parameter optimization

## Linear models

In [ ]:
class linearmodel(object):
    def __init__(self,data):
        self.data =data
        return
    def Linear_model(self,a4,a5):
        data = self.data
        DELTA = (2503 * np.exp(17.27 * data['Ta']/(data['Ta']+237.3)))/((data['Ta']+237.3)**2)
        gamma = 0.665*0.001* data['P']       
        gpp = a4 * data['SIF']
        T  = gpp * a5
        As = data['Rn'] - data['Ac']
        f = data['f']
        E = f * 1.26 *  DELTA * As /(DELTA + gamma)
        LE = T + E
        return [list(gpp),list(LE),list(T)]

In [ ]:
class spotpy_setup(object):
    def __init__(self,data,obs):
        self.data=data
        self.linearmodel= linearmodel(self.data)
        self.params = [
#                        spotpy.parameter.Uniform('a1',0,1),
#                        spotpy.parameter.Uniform('a2',0.9,1.1),
#                        spotpy.parameter.Uniform('a3',-1,0),
                       spotpy.parameter.Uniform('a4',5,50),
                       spotpy.parameter.Uniform('a5',3,50),
#                        spotpy.parameter.Uniform('a6',0,1)
                      ]
        self.obs=obs
#         self.OF=OF
        return
    def parameters(self):
        return spotpy.parameter.generate(self.params)
    def simulation(self,vector):
        simulations= self.linearmodel.Linear_model(vector[0],vector[1])
        return simulations
    def evaluation(self):
        observations=self.obs
        return observations

    def objectivefunction(self,simulation,evaluation):
        obj1=spotpy.objectivefunctions.nashsutcliffe(evaluation[0],simulation[0])# GPP
        obj2=spotpy.objectivefunctions.nashsutcliffe(evaluation[1],simulation[1])# ET
        obj = 1 - (0.5*obj1 + 0.5* obj2)
        return obj

def opt(data,rep=2000):
    opt_input = outlier_rev(data).dropna()
    obs = opt_input.loc[:,['GPP','LE']].T.values
    sampler=spotpy.algorithms.sceua(spotpy_setup(opt_input,obs), dbname='gs', dbformat='csv')
    sampler.sample(rep)
    results=sampler.getdata() 
    pars = spotpy.analyser.get_best_parameterset(results,maximize=False).tolist()[0]
    print(pars)
    out =  linearmodel(data).Linear_model(pars[0],pars[1])
    data['GPP_linear'],data['LE_linear'],data['T_linear'] = out[0],out[1],out[2]
    return data

In [ ]:
class wuemodel(object):
    def __init__(self,data):
        self.data =data

        return
    def vpd_model(self,a4,a5,a6):
        data = self.data

        DELTA = (2503 * np.exp(17.27 * data['Ta']/(data['Ta']+237.3)))/((data['Ta']+237.3)**2)
        vpd = (100 - data['Rh'])/100 * .6108 * np.exp(17.27*data['Ta']/(data['Ta']+237.3))
        gamma = 0.665*0.001* data['P']
#         if ptype == 'C4':
#             popt = r2(data['SIF'],data['GPP'])[2]
#             gpp = popt*data['SIF']# +  r2(data['SIF'],data['GPP'])[3]

#         else:
#             gpp = damm(data['SIF'],data['GPP'])
        gpp = a4 * data['SIF']
        T = a5 * gpp * vpd **a6
        As = data['Rn'] - data['Ac']       
        f = data['f']
        E = f * 1.26 *  DELTA * As /(DELTA + gamma)
        LE = E + T
        return [list(gpp),list(LE),list(T)]

class spotpy_setup(object):
    def __init__(self,data,obs):
        self.data=data
        self.wuemodel= wuemodel(self.data)
        self.params = [
#                        spotpy.parameter.Uniform('a1',0,1),
#                        spotpy.parameter.Uniform('a2',0.9,1.1),
#                        spotpy.parameter.Uniform('a3',-1,0),
                       spotpy.parameter.Uniform('a4',5,50),
                       spotpy.parameter.Uniform('a5',3,50),
                       spotpy.parameter.Uniform('a6',0.1,1),
#                        spotpy.parameter.Uniform('a7',0,1),
                      ]
        self.obs=obs
#         self.OF=OF
        return
    def parameters(self):
        return spotpy.parameter.generate(self.params)
    def simulation(self,vector):
        simulations= self.wuemodel.vpd_model(vector[0],vector[1],vector[2])
        return simulations
    def evaluation(self):
        observations=self.obs
        return observations

    def objectivefunction(self,simulation,evaluation):
        obj1=spotpy.objectivefunctions.nashsutcliffe(evaluation[0],simulation[0])#Biomass data
        obj2=spotpy.objectivefunctions.nashsutcliffe(evaluation[1],simulation[1])#Soil moisture data
        obj = 1 - (0.5*obj1 + 0.5* obj2)
        return obj

def opt_vpd(data,rep=2000):
    opt_input = outlier_rev(data).dropna()
    obs = opt_input.loc[:,['GPP','LE']].T.values
    sampler=spotpy.algorithms.sceua(spotpy_setup(opt_input,obs), dbname='gs', dbformat='csv')
    sampler.sample(rep)
    results=sampler.getdata() 
    pars = spotpy.analyser.get_best_parameterset(results,maximize=False).tolist()[0]
    print(pars)
    out = wuemodel(data).vpd_model(pars[0],pars[1],pars[2])
    data['GPP_wue'],data['LE_wue'],data['T_wue'] =  out[0],out[1],out[2]
    return data

## Conductance models

In [ ]:
def C4_model(data,a3,a4,a5):
    DELTA = (2503 * np.exp(17.27 * data['Ta']/(data['Ta']+237.3)))/((data['Ta']+237.3)**2)
#     kmol2ms = 44.6*273/(273+data['Ta'])*data['P']/101.3
    kmol2ms = 8.314510 * (273.16 + data['Ta'])/(data['P']*1000)
    gamma = 0.665*0.001* data['P']
    vpd = data['VPD']
    ql =  np.exp(-data['PAR'] *a3)
    j = a4 * data['SIF'] * ql
    gpp = j/4 # +  r2(data['SIF'],data['GPP'])[3]
    g = a5*gpp*data['Rh']/100/data['CO2'] 
#     g = 0.00001 + 1.6*(1 + a5/data['VPD']**0.5)*gpp/data['CO2']
    T = (DELTA*data['Ac'] + data['rho_a']*Cp*vpd*data['Ga'])/(DELTA + gamma*(1 + data['Ga']/(g*kmol2ms)))
#     T = data['rho_a']*Cp*vpd/gamma/(1/data['Ga']+1/(g*kmol2ms))
    As = data['Rn'] - data['Ac']
    f = data['f']
#     f = data['MS']
    E = f * 1.26 *  DELTA * As /(DELTA + gamma)
    LE = T + E
    return [list(LE),list(gpp),list(T),list(g)]

In [ ]:
class gsmodel(object):
    def __init__(self, data):
        self.data =data
        return
    
class spotpy_setup(object):
    def __init__(self,data,obs):
        self.data=data
        self.gsmodel= gsmodel(self.data)
        self.params = [
#                        spotpy.parameter.Uniform('a1',0,1),
#                        spotpy.parameter.Uniform('a2',0.9,1.1),
                       spotpy.parameter.Uniform('a3',0,0.001),
                       spotpy.parameter.Uniform('a4',10,300),
                       spotpy.parameter.Uniform('a5',2.5,8.8),
#                        spotpy.parameter.Uniform('a6',0,1),
#                        spotpy.parameter.Uniform('a7',0.5,1)
                      ]
        self.obs=obs
#         self.OF=OF
        return
    def parameters(self):
        return spotpy.parameter.generate(self.params)
    def simulation(self,vector):
        simulations= C4_model(self.data,vector[0],vector[1],vector[2])
        return simulations
    def evaluation(self):
        observations=self.obs
        return observations

    def objectivefunction(self,simulation,evaluation):
        obj1=spotpy.objectivefunctions.nashsutcliffe(evaluation[0],simulation[0])#Biomass data
        obj2=spotpy.objectivefunctions.nashsutcliffe(evaluation[1],simulation[1])#Soil moisture data
        obj = 1 - (0.6*obj1 + 0.4* obj2)
        return obj

def opt_C4(data,rep=5000):
    opt_input = outlier_rev(data).dropna()
    obs = opt_input.loc[:,['LE','GPP']].T.values
    sampler=spotpy.algorithms.sceua(spotpy_setup(opt_input,obs), dbname='gs', dbformat='csv')
    sampler.sample(rep)
    results=sampler.getdata() 
    pars = spotpy.analyser.get_best_parameterset(results,maximize=False).tolist()[0]
    print(pars)
    out = C4_model(data,pars[0],pars[1],pars[2])
    data['LE_gs'],data['GPP_gs'],data['T_gs'],data['g_gs'] = out[0],out[1],out[2],out[3]
    return data

In [ ]:
daman = opt_C4(daman)
huailai = opt_C4(huailai)

In [ ]:
class Gu_EB(object):
    def __init__(self,data):
        self.data = data
        self.Gamma = data['Gamma']
        self.ca = data['CO2']
        return
    def g_C3(self,J,vpd,lamb = None):
        Gamma = self.Gamma
        ca = self.ca
        a = 1.6
        vpd = vpd/self.data['P']
        lamb = lamb
        a1  = J/4
        a2 = 2*Gamma
        g = -a1 * (a2 - ca + 2*Gamma)/(a2 + ca)**2 + \
            (1.6*vpd*lamb*(a1)**2*(ca - Gamma)*(a2 + Gamma)*(a2 + ca -3.2*vpd*lamb)**2*(a2+ca-1.6*vpd*lamb))**0.5/(1.6*vpd*lamb*(a2 + ca)**2*(a2 + ca - 1.6*vpd*lamb))
        return g
    
    def GPP_C3(self,J,g):
        Gamma = self.Gamma
        ca = self.ca
        a1  = J/4
        a2 = 2*Gamma

        gpp = 0.5 * (a1 + (a2 + ca)*g-((a1+g*(a2-ca))**2+4*g*(a1*Gamma+a2*g*ca))**0.5)
        return gpp

In [ ]:
def C3_model(data,a3,a4,a5):
#     kmol2ms = 44.6*273.15/(273.16+data['Ta'])*data['P']/101.3
    kmol2ms = 8.314510 * (273.16 + data['Ta'])/(data['P']*1000)
    
    gmodel = Gu_EB(data)
    
    DELTA = (2503 * np.exp(17.27 * data['Ta']/(data['Ta']+237.3)))/((data['Ta']+237.3)**2)   
    
    gamma = 0.665*0.001* data['P']
    
    vpd = data['VPD']
    ql =  np.exp(-data['PAR'] *a3)
    j = a4 * data['SIF'] *ql
    
    lamb = a5 #* (621/(28.96 * 0.287 * (273.15+data['Ta']) * kmol2ms))
    g =  gmodel.g_C3(j,vpd,lamb = lamb)

    gpp = gmodel.GPP_C3(j,g)
    T = (DELTA*data['Ac'] + data['rho_a']*Cp*vpd*data['Ga'])/(DELTA + gamma*(1 + data['Ga']/(1.6*g*kmol2ms)))
#     T = data['rho_a']*Cp*vpd/gamma/(1/data['Ga']+1/(1.6*g/kmol2ms))
    As = data['Rn'] - data['Ac']
    f = data['f']
    E = f * 1.26 *  DELTA * As /(DELTA + gamma)
    LE = T + E
    return [list(LE),list(gpp),list(T),list(g)]

In [ ]:
class gsmodel(object):
    def __init__(self, data):
        self.data =data
        return
    
class spotpy_setup(object):
    def __init__(self,data,obs):
        self.data=data
        self.gsmodel= gsmodel(self.data)
        self.params = [
#                        spotpy.parameter.Uniform('a1',4,8),
#                        spotpy.parameter.Uniform('a2',0.5,1.2),
                       spotpy.parameter.Uniform('a3',0,0.01),
                       spotpy.parameter.Uniform('a4',10,300),
                       spotpy.parameter.Uniform('a5',0,2500),
#                        spotpy.parameter.Uniform('a6',0,1)
                      ]
        self.obs=obs
#         self.OF=OF
        return
    def parameters(self):
        return spotpy.parameter.generate(self.params)
    def simulation(self,vector):
        simulations= C3_model(self.data,vector[0],vector[1],vector[2])
        return simulations
    def evaluation(self):
        observations=self.obs
        return observations

    def objectivefunction(self,simulation,evaluation):
        obj1=spotpy.objectivefunctions.nashsutcliffe(evaluation[0],simulation[0])#Biomass data
        obj2=spotpy.objectivefunctions.nashsutcliffe(evaluation[1],simulation[1])#Soil moisture data
        obj = 1 - (0.5*obj1 + 0.5* obj2)
        return obj
#         return [obj1,obj2]

def opt_C3(data,rep=200):
    
    opt_input = outlier_rev(data).dropna()
    obs = opt_input.loc[:,['LE','GPP']].T.values

    sampler=spotpy.algorithms.sceua(spotpy_setup(opt_input,obs), dbname='gs', dbformat='csv')
    sampler.sample(rep)
    results=sampler.getdata() 
    pars = spotpy.analyser.get_best_parameterset(results,maximize=False).tolist()[0]
    print(pars)
    out = C3_model(data,pars[0],pars[1],pars[2])
    data['LE_gs'],data['GPP_gs'],data['T_gs'],data['g_gs'] = out[0],out[1],out[2],out[3]
    data['GPP_gs'] = data['GPP_gs']
    return data

In [ ]:
niwot = opt_C3(niwot)
harvard = opt_C3(harvard)

In [ ]:
niwot.to_csv('nw_winter.csv')

In [ ]:
data = pd.concat([daman,huailai,niwot,harvard]).dropna()

In [ ]:
data.to_csv('LE.csv')